<a href="https://colab.research.google.com/github/dudaspm/IST210/blob/master/StartingSQL/LearningSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is setup to let you learn about SQL. 

First, let's get the data we will be working with today. 

#### Seattle Weather
from: https://vega.github.io/vega-datasets 

In [38]:
#Used to make requests
import urllib

f = urllib.request.urlopen("https://vega.github.io/vega-datasets/data/seattle-weather.csv")
text = str(f.read().decode('utf-8'))

for line in text.split("\n")[0:5]:
  print (line.split(","))

['date', 'precipitation', 'temp_max', 'temp_min', 'wind', 'weather']
['2012/01/01', '0.0', '12.8', '5.0', '4.7', 'drizzle']
['2012/01/02', '10.9', '10.6', '2.8', '4.5', 'rain']
['2012/01/03', '0.8', '11.7', '7.2', '2.3', 'rain']
['2012/01/04', '20.3', '12.2', '5.6', '4.7', 'rain']


Let's create a database and table to house this data. For this exercise, we will be using SQLite. 

When it comes to using SQLite in Python, we need to first activate the SQLite library. We do this by using import.

In [0]:
import sqlite3 # Once imported in a single notebook, it is available to us throughout the rest of the notebook. 

Now, we connect to the database. 

* But wait.. we didn't create the database yet. How is it going to connect? 

Connecting will create the database, if there is no database available by that name. 

In [0]:
# connect to database, if not there, create it.
con = sqlite3.connect('WeatherData') 

Great! The database is now created. 

* How do we interact with it? 

We create a *cursor* to the database.

In [0]:
cursor = con.cursor()

The first SQL that we need is to create the weather table. This is done using CREATE and listing the attributes, their attribute types, and any primary keys.

We will use our *cursor* to *execute* a specific command. This case CREATE.

**IMPORTANT** to make sure we handling things correctly, there is an additional step. We must *commit* these changes to the database.

In [0]:
cursor.execute('''create table IF NOT EXISTS seattleWeather (date text PRIMARY KEY, precipitation real, temp_max real, temp_min real, wind real, weather text)''')
con.commit()



*   INTEGER
  * whole number
*   REAL
  * float value
*   TEXT
  * characters, strings
*   BLOB
  * Not recommmend, but images/videos


Next, we need to INSERT data into our table. 

This first time through, we will be just inserting a single value.

In [0]:
cursor.execute('''insert into seattleWeather values ('2012/01/01', 0.0, 12.8, 5.0, 4.7, 'drizzle' );''')
con.commit()

Now, let's check and make sure it is there in our table.

We will be using a SELECT statement. 

Reminder, * - means wildcard.

In [44]:
cursor.execute('''select * from seattleWeather;''')
print (cursor.fetchall())

[('2012/01/01', 0.0, 12.8, 5.0, 4.7, 'drizzle')]


At this point, we should see this:

[('2012/01/01', 0.0, 12.8, 5.0, 4.7, 'drizzle')]

* What happens if we re-run the insert statement?

* Why does this happen?

Finally, we might want to delete or DROP the table. Remember, this will **permanently** delete **ALL** data from the table. 

In [0]:
cursor.execute('''drop table seattleWeather;''')
con.commit()

* What would happen if we would run a select statement now?

OK, let's create the table again and put all the data in this time. 

I am also adding a "IF NOT EXISTS" clause. This make sure if the table is there, we don't try to create it.

In [0]:
cursor.execute('''create table IF NOT EXISTS seattleWeather (date text PRIMARY KEY, precipitation real, temp_max real, temp_min real, wind real, weather text)''')
con.commit()
for line in text.split("\n")[1:-1]:
  rowOfData = line.split(",")
  date = rowOfData[0]
  precipitation = rowOfData[1] 
  temp_max = rowOfData[2] 
  temp_min = rowOfData[3] 
  wind = rowOfData[4] 
  weather = rowOfData[5] 
  # I like to create a string statement for my inserts. It gives me a chance to see the insert statement before I commit changes.
  insertStatement = """insert into seattleWeather values ("%s",%s,%s,%s,%s,"%s")""" % (date,precipitation,temp_max,temp_min,wind,weather)
  cursor.execute(insertStatement)
  con.commit() # remember to commit!


We are going to limit our responses from the database by using **limit**

In [47]:
cursor.execute('''select * from seattleWeather limit 5;''')
for row in cursor.fetchall():
  print (row)

('2012/01/01', 0.0, 12.8, 5.0, 4.7, 'drizzle')
('2012/01/02', 10.9, 10.6, 2.8, 4.5, 'rain')
('2012/01/03', 0.8, 11.7, 7.2, 2.3, 'rain')
('2012/01/04', 20.3, 12.2, 5.6, 4.7, 'rain')
('2012/01/05', 1.3, 8.9, 2.8, 6.1, 'rain')


Let's try a select statement that select only data that is weather = "rain". 

This is when use the **WHERE** statement.

In [48]:
cursor.execute('''select * from seattleWeather where weather = "rain" limit 10;''')
for row in cursor.fetchall():
  print (row)

('2012/01/02', 10.9, 10.6, 2.8, 4.5, 'rain')
('2012/01/03', 0.8, 11.7, 7.2, 2.3, 'rain')
('2012/01/04', 20.3, 12.2, 5.6, 4.7, 'rain')
('2012/01/05', 1.3, 8.9, 2.8, 6.1, 'rain')
('2012/01/06', 2.5, 4.4, 2.2, 2.2, 'rain')
('2012/01/07', 0.0, 7.2, 2.8, 2.3, 'rain')
('2012/01/09', 4.3, 9.4, 5.0, 3.4, 'rain')
('2012/01/10', 1.0, 6.1, 0.6, 3.4, 'rain')
('2012/01/21', 3.0, 8.3, 3.3, 8.2, 'rain')
('2012/01/22', 6.1, 6.7, 2.2, 4.8, 'rain')
